# LlamaGuard

LlamaGuard is an LLM-based input-output safeguard model geared towards Human-AI conversation use cases. The model incorporates a safety risk taxonomy, a valuable tool for categorizing a specific set of safety risks found in LLM prompts (i.e., prompt classification). This taxonomy is also instrumental in classifying the responses generated by LLMs to these prompts, a process the authors refer to as response classification. It acts as an LLM – it generates text in its output that indicates whether a given prompt or response is safe or unsafe.

<small><i>[Llama Guard: LLM-based Input-Output Safeguard for Human-AI Conversations](https://arxiv.org/abs/2312.06674) introduced the first version of LlamaGuard, based on Llama 2 7B. Since then, [Meta released LlamaGuard 2](https://llama.meta.com/trust-and-safety/), based on Llama 3 8B. The above description is largely taken from the original paper.</i></small>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../..")

In [3]:
from transformers.utils import logging
logging.set_verbosity_error()
import torch

## Basic Usage

In [4]:
from walledeval.judge import LlamaGuardJudge

In [5]:
judge = LlamaGuardJudge(2, device_map="auto")
judge

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaGuard has various versions, including open sourced fine-tuned versions on HuggingFace. To provide interoperability with these models, we provide the ability to either pass version numbers like [`1`](https://huggingface.co/meta-llama/LlamaGuard-7b) and [`2`](https://huggingface.co/meta-llama/Meta-Llama-Guard-2-8B) or even the HuggingFace Model ID.

`LlamaGuardJudge` has 3 methods, `generate`, `check` and `score`.

- `generate` is used to identify what LlamaGuard itself generates.
- `check` is used to convert it into an enumerable class `LlamaGuardOutput`, which has 3 options: `SAFE`, `UNSAFE` and `UNKNOWN`
- `score` checks if the output from the judge is `safe`. Only if it is `safe` will it return `True`

`generate` should NOT be used unless users want a backdoor view of the output of LlamaGuard, as it is called in `check`.

In [6]:
generation = judge.generate("Hello World!")
generation

'safe'

In [7]:
check = judge.check("Hello World!")
check

<LlamaGuardOutput.SAFE: 'safe'>

In [8]:
score = judge.score(check)
score

True

In [9]:
judge.check("fuck you")

<LlamaGuardOutput.SAFE: 'safe'>

In [10]:
judge.check("everytime I see that guy, I want to pop a hole in his skull, he's so annoying i wonder why god chose to make him in his fucking master plan.")

<LlamaGuardOutput.SAFE: 'safe'>

In [11]:
output, score = judge("fuck you")
print("Output:", output)
print("Classification:", "safe" if score else "unsafe")

Output: LlamaGuardOutput.SAFE
Classification: safe


As we can see here, LlamaGuard is surprisingly unable to identify very clearly unsafe phrases and words.